In [167]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm import tqdm
mpl.rcParams['figure.dpi'] = 300
mpl.rc('font', size=12)

from numba import njit

# Voter model with mutation

In [168]:
@njit
def init_grid(L):
    '''Initialises an L by L grid with numbers drawn randomly from (0, 1)'''
    return np.random.rand(L, L)

In [169]:
@njit
def get_4_neighbors(i, j, L):
    '''Finds upper, lower, left and right neighbor of a site in a K by K grid (Von Neumann neighborhood)'''
    neighbors = []
    # Up, down, right, left
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  
    for di, dj in directions:
        # Apply periodic boundary conditions
        neighbor_i = int((i + di) % L)
        neighbor_j = int((j + dj) % L)
        neighbors.append((neighbor_i, neighbor_j))
    return neighbors

In [170]:
import itertools

def moore_neighborhood_directions(R):
    # Find coordinates of all neighbors within range R
    directions = list(itertools.product(range(-R, R + 1), range(-R, R + 1)))
    del directions[int(len(directions)/2)]

    return directions

In [171]:
@njit
def get_9_neighbors(i, j, L, moore_directions):
    '''Finds all surrounding neighbors of a site in a L by L grid (Moore neighborhood)'''
    neighbors = []

    for di, dj in moore_directions:
        # Apply periodic boundary conditions
        neighbor_i = int((i + di) % L)
        neighbor_j = int((j + dj) % L)
        neighbors.append((neighbor_i, neighbor_j))
    return neighbors

In [172]:
import math

@njit
def get_neighbors_fattail(i, j, L, eta, moore_directions):
    ''' Computes fat-tailed dispersal kernel
    Parameters:
    - i: x-coordinate of center
    - j: y-coordinate of center
    - L: length scale or width of kernel
    - R: distance between parent and offspring
    - eta: defines "fatness" of the distribution

    Returns:
    - neighbors: list of all surrounding neighbors within range R
    - probabilities: list of all probabilities taken from a fat-tailed distribution given distance between center and neighbor
    '''

    neighbors = []
    probabilities = []

    # Find coordinates of neighbors within range R
    for di, dj in moore_directions:
        # Apply periodic boundary conditions
        neighbor_i = int((i + di) % L)
        neighbor_j = int((j + dj) % L)
        neighbors.append((neighbor_i, neighbor_j))

        # Calculate Euclidean distance between center and neighbor
        dist = math.dist([i, j], [neighbor_i, neighbor_j])

        # Calculate fat-tail probability
        K = ((eta + 2) / (2 * np.pi * L**2)) * (1 + ((dist)/L)**2 )**(eta/2)
        probabilities.append(K)
        
    probabilities = np.array(probabilities) / sum(probabilities)
    return neighbors, probabilities

In [173]:
from scipy import stats
from sklearn.metrics.pairwise import euclidean_distances

def get_neighbors_gaussian(i, j, L, moore_directions):
    ''' Computes Gaussian dispersal kernel
    Parameters:
    - i: x-coordinate of center
    - j: y-coordinate of center
    - L: length scale or width of kernel
    - R: distance between parent and offspring

    Returns:
    - neighbors: list of all surrounding neighbors within range R
    - probabilities: list of all probabilities taken from a Gaussian distribution given distance between center and neighbor
    '''

    neighbors = []
    probabilities = []

    # Find coordinates of all neighbors within range R
    for dir in moore_directions:
        neighbors.append(tuple(map(lambda x, y: (x + y) % L, dir, (i, j))))

    dist = euclidean_distances(neighbors, [[i,j]])
        
    for d in dist:
        probabilities.append(float(stats.norm.pdf(d, 0, 1)))
    
    probabilities = np.array(probabilities) / sum(probabilities)

    return neighbors, probabilities

In [174]:
import random

# @njit
def voter_model(grid_0, alpha, n_iters, kernel='nearest', eta=None, R=None):
    '''Run experiment with the voter model
    Inputs:
    grid_0 (numpy array): Initial grid
    alpha (float): Value of alpha parameter
    n_iters (int): number of monte carlo steps
    eta (float/int): defines "fatness" of the distribution
    L (int): length scale or width of kernel
    R (int): distance between parent and offspring
    
    Returns:
    cur_grid (numpy array): Grid after n_iter iterations
    num_species (list): Contains amount of different species at each tenth iteration
    '''
    # Create list to store number of unique species
    num_species = []
    cur_grid = np.copy(grid_0)

    height, width = cur_grid.shape

    xs = np.random.choice(np.arange(0, width), size=n_iters)
    ys = np.random.choice(np.arange(0, height), size=n_iters)

    if kernel in ['fat tail', 'gaussian', 'nearest_moore']:    
        moore_directions = moore_neighborhood_directions(R)
        
    # Update grid n_iter times
    for i in tqdm(range(n_iters)):
        if random.random() < alpha:
            cur_grid[xs[i], ys[i]] = random.random()
        # Set species in cell to that in one of its 4 neighbors with equal probability
        else:
            if kernel == 'nearest':
                neighbors = get_4_neighbors(xs[i], ys[i], L)
                neighbor_idx = np.random.choice(len(neighbors))
                neighbor = neighbors[neighbor_idx]
            elif kernel == 'nearest_moore':
                neighbors = get_9_neighbors(xs[i], ys[i], L, moore_directions)
                neighbor_idx = np.random.choice(len(neighbors))
                neighbor = neighbors[neighbor_idx]
            elif kernel == 'fat tail':
                neighbors, probabilities = get_neighbors_fattail(xs[i], ys[i], L, eta, moore_directions)
                neighbor_idx = np.random.choice(len(neighbors), p=probabilities)
                neighbor = neighbors[neighbor_idx]
            elif kernel == 'gaussian':
                neighbors, probabilities = get_neighbors_gaussian(xs[i], ys[i], L, moore_directions)
                neighbor_idx = np.random.choice(len(neighbors), p=probabilities)
                neighbor = neighbors[neighbor_idx]

                
            new_type = cur_grid[neighbor]
            cur_grid[xs[i], ys[i]] = new_type
            
        # Save amount of species every tenth iteration
        if i % (n_iters//100) == 0:
            num_species.append(len(np.unique(cur_grid)))
    return cur_grid, num_species

In [175]:
def sa_curve(grid):
    height, width = grid.shape
    
    n_centers = 10
    centers_x = np.random.choice(np.arange(0, width), n_centers) + width
    centers_y = np.random.choice(np.arange(0, height), n_centers) + height

    areas = []
    species = []
    
    torus_grid = np.vstack((grid, grid, grid))
    torus_grid = np.hstack((torus_grid, torus_grid, torus_grid))
    
    torus_grid.shape
    
    for i, (x, y) in enumerate(zip(centers_x, centers_y)):
        cur_species = []
        for j in range(width//2):
            cur_species.append(len(np.unique(torus_grid[x-j:x+j+1, y-j:y+j+1])))
            if i == 0:
                areas.append((j+1)**2)
        species.append(cur_species)
    
    return areas, species